In [1]:
import os
from openai import OpenAI
import openai
from tool import videoreader
import json

In [2]:
dimension = 'imaging_quality'
from tool.framework import critic_agent
agent1 = critic_agent()

In [3]:
data_prepath = r'D:\Astudying\VideoEval\data4dimensions'
# data_prepath = "../../data4dimensions/"
with open("./Human_anno/{}.json".format(dimension)) as f:
    human_anno = json.load(f)

eval_path ="./GPT4o_eval_results/{}/{}_llmeval_7inputsfewshot.json".format(dimension,dimension)
with open(eval_path,'r') as f:
    eval_results = json.load(f)

critic_evalph = "./GPT4o_eval_results/imaging_quality/imaging_quality_multillmeval.json"
with open(critic_evalph,'r') as f:
    critic_eval = json.load(f)

In [4]:
# l1 = list(range(0,len(human_anno),3))
# l2 = list(range(161,len(human_anno),3))
l1 = [0]
l = l1
for i in l:
    if str(i) not in critic_eval:
        critic_eval[str(i)] = {}

In [11]:
# skip_index = list(range(0, len(human_anno),5))
model2message = {
'cogvideox5b':"12 frames from cogvideox5b\n",
'kling':"10 frames from kling \n ", 
'gen3': "10 frames from gen3 \n",
'videocrafter2':"4 frames from videocrafter2",
'pika':"7 frames from pika ",
'show1':"8 frames from show1 ",
'lavie':"5 frames from lavie ",
}

for i in l:        

    examplemodels = [x for x in model2message.keys()]
    eval_res = eval_results[str(i)] 
    frames = videoreader.process_video(data_prepath,human_anno[i]['videos'],2)
    
    # frames = videoreader.process_video(data_prepath,human_anno[i]['videos'],2)
    # videoreader.display_base64_images(frames['kling'])

    prompten = human_anno[i]['prompt_en']
    # question = human_anno[i]['question_en']
    # subject = human_anno[i]['subject_en']
    # scene = human_anno[i]['scene_en']
    # objet = human_anno[i]['object']

    try:
        response = agent1.client.chat.completions.create(
        model=agent1.model_name, 
        messages=[
        {
        "role": "system", "content":
        agent1.system_message,
        }
        ,
        {
        "role": "user", "content": [
        "According to  your **Goals**in system meassage, there are examples and evaluation results from video generation models.\n",
        *[item for examplemodel in examplemodels for item in [
            "This example is from model '{}' \n".format(examplemodel),
            {"type": "image_url", "image_url": {"url": f'data:image/jpg;base64,{frames[examplemodel][0]}', "detail": "low"}}
        ]],                 
        "The evaluation results of these examples are as follows:\n",
        *map(lambda x: f"model: {x}, evaluation result: {eval_res[x]}\n", examplemodels),
        
        "Assuming there are videos you think should score 'x',provide your analysis and explanation in the output format as follows:\n",
        *map(lambda x: f"- {x}: should be scored x,because...\n", examplemodels),

            ],
            }
        ],
        temperature=0,
        )
        print(response.choices[0].message.content) 
        critic_eval[str(i)] = response.choices[0].message.content.replace('\n\n','\n')
    except Exception as e:
        print(f"An error occurred: {e}")
        critic_eval[str(i)] = "An error occurred"
        

Here’s the analysis and evaluation of the video frames based on the provided examples and evaluation results:

- **cogvideox5b**: should be scored **4**, because the frames exhibit good clarity with vibrant colors and minimal distortions. The brightness is well-balanced, contributing to a high-definition quality overall.

- **kling**: should be scored **4**, because the frames demonstrate good clarity and high definition. The minimal distortions and well-balanced brightness enhance the viewing experience, resulting in high video quality.

- **gen3**: should be scored **3**, because while the frames show good clarity and vibrant colors, there is noticeable overexposure in some areas that detracts from the overall quality. The details are well-defined, but the slight overexposure impacts the viewing experience.

- **videocrafter2**: should be scored **4**, because the frames display high-definition clarity with vibrant colors and minimal noise. The brightness is well-balanced, providing 

In [6]:
# import json
# # 使用 json 保存字典
# with open(file_path, "w") as f:
#     json.dump(s, f,indent=4)